In [3]:
import random

from enum import Enum, unique
from collections import namedtuple
from typing import List, Union


CardType = Enum('CardType',[
    'HERO',
    'WEAPON',
    'HERO_EVENT',
    'HERO_UPGRADE',
    'HERO_POWER',
    'CHECK_MISS',
    'CHECK_HIT',
    'HERO_SUPPORT',
])


HeroActions = Enum('HeroActions',[
    'ATTACK',
    'MOVE',
    'ENGAGE',
    'DISENGAGE',
    'INTERACT',
    'CLIMB',
    'REST',
    'SECOND_CHANCE',
])


GameState = Enum('GameState',[
    'DRAFTING',
    'DEPLOYMENT',
    'START_OF_ROUND',
    'DETERMINE_INITIATIVE',
    'DRAW_CARDS',
    'PLAY_ROUND',
    'POINTS_CHECK',
    'END_OF_ROUND',
])


HeroState = Enum('HeroState',[
    'ACTIVE',
    'STUNNED',
    'FROZEN',
    'DISABLED'
])
    

Position = namedtuple('Position', ['x', 'y'])
ActionCost = namedtuple('ActionCost', ['cards','action_points','action_range'])



class Card:
    def __init__(self, card_type:CardType, name:str, owner:str = None):
        self._type = card_type
        self._name = name
        self._owner = owner

    @property
    def name(self) -> str:
        return self._name

    @property
    def card_type(self) -> CardType:
        return self._type

    @property
    def owner(self) -> str:
        return self._owner

    @owner.setter
    def owner(self, value:str):
        self._owner = value


class Deck:
    def __init__(self, cards:List[Card] = []):
        self._cards:List[Card] = cards if cards else []

    def queue_cards(self, cards: List[Card]):
        if cards:
            self._cards += cards

    def push_cards(self, cards: List[Card]):
        if cards:
            self._cards = cards + self._cards

    def merge(self, other):
        """
        """
        self._cards += other._cards

    def pop_cards(self, count:int=1) -> List[Card]:
        count = min(max(0, count), len(self._cards))
        cards = self._cards[0:count]
        if count:
            self._cards = self._cards[count:]
        return cards

    def shuffle(self):
        random.shuffle(self._cards)

    @property
    def count(self):
        return len(self._cards)

    def __iter__(self):
        return self._cards.__iter__()


class WeaponBase:
    def __init__(self, name:str, area:int, damage:int):
        self._name = name
        self._area = area
        self._damage = damage

    def can_attack(self, source, target) -> bool:
        return False

    def attack(self, source, target) -> bool:
        """
        Attacks a target, if all requirements are met
        """
        if self.can_attack(source, target):
            self._perform_attack(source, target)
            return True
        return False

    def _perform_attack(self, source, target):
        """
        Performs the attack if all requirements are met
        """
        pass

    def on_disabled(self):
        """
        Perform clean-up actions specific to the weapon
        """
        pass

    
class HeroBase:
    def __init__(self, name:str, life:int, desc:str, weapons=List[WeaponBase]):
        self._name = name
        self._life = life
        self._desc = desc
        self._state = HeroState.ACTIVE
        self._action_points = 0
        self._weapons = weapons
        self.position = Position(x=-1, y=-1)
        
    @property
    def name(self) -> str:
        return self._name
    
    @property
    def description(self) -> str:
        return self._desc
    
    @property
    def life(self) -> int:
        return self._life

    @property
    def is_active(self) -> bool:
        return self._state != HeroState.DISABLED

    @property
    def state(self) -> HeroState:
        return self._state

    @property
    def action_points(self) -> int:
        return self._action_points

    @property
    def weapons(self) -> List[WeaponBase]:
        return self._weapons

    @property
    def position(self) -> Position:
        return self._position

    @position.setter
    def position(self, value:Position):
        self._position = value
        if self._position.x < 0 or self._position < 0:
            self._state = HeroState.DISABLED
            self._on_disabled()
    
    def consume_life(self, points:int) -> HeroState:
        self._life = max(0, self._life - points)
        if self._life <= 0:
            self._state = HeroState.DISABLED
            self._on_disabled()
        return self._state

    def _disable(self):
        if not self.is_active:
            return
        self._state = HeroState.DISABLED
        self._on_disabled()
        if not self._weapons:
            return
        for weapon in self._weapons:
            weapon.on_disabled()

    def _on_disabled(self):
        """
        Perform clean-up actions specific to the hero
        """
        pass


class Skill:
    def __init__(self, name:str):
        self._name = name
    
    @property
    def name(self):
        return self._name

    def on_game_state_changed(self, old_state:GameState, new_state:GameState):
        pass
    
    def consume_requirements(self, hero:HeroBase):
        pass


class Hero(HeroBase):
    def __init__(self, name:str, life:int, desc:str="", weapons=List[WeaponBase]):
        super().__init__(name=name, life=life, desc=desc)



ACTION_COSTS = {
    HeroActions.ATTACK: ActionCost(cards=0, action_points=1, action_range=-1), # A -1 range means that the range is dependent
    HeroActions.MOVE: ActionCost(cards=0, action_points=1, action_range=3),
    HeroActions.INTERACT: ActionCost(cards=1, action_points=0, action_range=-1),
    HeroActions.CLIMB: ActionCost(cards=1, action_points=0, action_range=-1),
    HeroActions.SECOND_CHANCE: ActionCost(cards=1, action_points=0, action_range=-1),
    HeroActions.REST: ActionCost(cards=0, action_points=2, action_range=-1),
    HeroActions.ENGAGE: ActionCost(cards=2, action_points=0, action_range=1),
    HeroActions.DISENGAGE: ActionCost(cards=3, action_points=0, action_range=-1),
}



"""
Weapons
"""
class NaturesTouch(WeaponBase):
    def __init__(self):
        super().__init__(name="Nature's Touch", area=1, damage=1)

    def can_attack(self, source:Hero, target:Hero) -> bool:
        return False


"""
Heroes
"""

HEROES = [
    Hero("Ivy", life=7),
    Hero("Sally", life=8),
    Hero("Wonder", life=6),
    Hero("Harry", life=6),
    Hero("Lara", life=6),
    Hero("Bat", life=7),
    Hero("Daeny", life=6),
    Hero("Quinn", life=6),
    Hero("Super", life=6),
    Hero("Ripley", life=6),
    Hero("Alice", life=6),
    Hero("Furiosa", life=7),
]